In [1]:
import sys                                                                         # Lo vamos a hacer con TensorFLow pero dentro de este tenemos la posibilidad de usar Keras.
import os                                                                          # Vamos a importar las librerias que nos van a permitir movernos entre los directorios de nuestro sistema.
import tensorflow as tf
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator         # Libreria que nos va a pre-procesar las imagenes para entrenar a nuestro algoritmo:
from tensorflow.python.keras import optimizers                                     # Importo los optimizadores:
from tensorflow.python.keras.models import Sequential                              # Importo la libreria que nos permite hacer redes neurnales secueniales (Cada una de las capas está ordenada):
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D             # Importo las capas en las cuales vamos a estar haciendo nuestras convoluciones y max-pooling:
from tensorflow.python.keras import backend as K                                   # Importo la libreria que nos va a ayudar a que si hay una sesión de queras que está corriendo en segundo plano, matarl

tf.logging.set_verbosity(tf.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
K.clear_session() 

In [3]:
data_entrenamiento = r'C:\Users\JOSEA\Downloads\data\train'                           # La "r" al principio de la string la va a transformar en uns RAW string para que no detecte el comando especial "\".
data_validacion = r'C:\Users\JOSEA\Downloads\data\validation'

In [4]:
epocas = 20                                                                        # Veces que vamos a iterar nuestro DataSet.
altura, longitud = 100, 100                                                        # Tamaño al que vamos a procesar nuestras imágenes.
batch_size = 32                                                                    # Número de imágenes que vamos a enviar para procesar en cada uno de las iteraciones. (Evitar sobrecarga de RAM).
pasos = 1000                                                                       # Número de veces que se va a procesar la información en cada una de las épocas.
pasos_validacion = 200                                                             # Al final de cada época se van a ejecutar 200 pasos con nuestro DataSet de validación (Comprobación).
filtrosConv1 = 32
filtrosConv2 = 64                                                                  # Número de filtros que va a haber en cada convolución. Profundidad de las capas ocultas.
tamano_filtro1 = (3,3)
tamano_filtro2 = (2,2)                                                             # Filtro(altura x longitud)
tamano_pool = (2,2)                                                                # Tamaño del filtro en MaxPooling
clases = 2                                                                         # "gato", "perro", "gorila"
lr = 0.0005  

In [5]:
entrenamiento_datagen = ImageDataGenerator(
    rescale = 1./255,                          # Transforma cada pixel de un RANGO de 0-255 a un rango de 0-1 (Downnscale)
    shear_range = 0.3,                         # Va a INCLINAR y rotar cada imagen para que el algoritmo aprenda como es el objeto desde todas las perspectivas
    zoom_range = 0.3,                          # Va a hacer ZOOM a cada imagen para entrenar al algoritmo de forma alterna.
    horizontal_flip = True                     # Va a INVERTIR la imagen.
)

validacion_datagen = ImageDataGenerator(
    rescale = 1./255                           # Solo REESCALO estas imágenes de test para comparar con los resultados de entrenamiento.
)


In [6]:
imagen_entrenamiento = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,                        # Va a entrar al directorio "data".
    target_size = (altura, longitud),          # Va a preprocesar todas las imágenes que se encuentre a una altura y longitud (definidas arriba).
    batch_size = batch_size,                   # Va a tomar una cantidad de 32 imágenes para cada iteración.
    class_mode = 'categorical'                 # La clasificación va a ser categórica ["perro","gato","gorila"]
)

imagen_validacion = validacion_datagen.flow_from_directory(
    data_validacion,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical'
)


Found 2048 images belonging to 2 classes.
Found 832 images belonging to 2 classes.


In [7]:
cnn = Sequential()                                                                                                      # Le vamos a decir que la Red sea seuencial, es decir, varias capas apiladas entre ellas.

cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding='same', input_shape=(altura, longitud,3), activation='relu')) # Convolución(Filtros, Tamaño_Filtro, Filtro_esquinas, Tamaño_Imagen, Función_de_activación=RELU).
cnn.add(MaxPooling2D(pool_size = tamano_pool))                                                                          # Después de la capa de Convolución vamos a tener una capa de MaxPooling que va a tener un tamaño de (2,2) pixels.

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding='same', activation='relu'))                                 # Convolución(Filtros, Tamaño_Filtro, Filtro_esquinas, Función_de_activación=RELU).
cnn.add(MaxPooling2D(pool_size = tamano_pool))                                                                          # Después de la capa de Convolución vamos a tener una capa de MaxPooling que va a tener un tamaño de (2,2) pixels.

cnn.add(Flatten())                                                                                                      # Vamos a transformar la imagen que es muy profunda y pequeña a una muy grando plana de una sola dimensión.
cnn.add(Dense(256, activation = 'relu'))                                                                                # Añade una capa de 256 neuronas donde van a estar todas las neuronas de las capas anteriores. Función de activación es relu.
cnn.add(Dropout(0.5))                                                                                                   # Le voy a "apagar" el 50% de las neuronas para que no esté demasiado ajustado. No quiero que encuentre un único camino para "perros" sino que encuentre varios.
cnn.add(Dense(clases, activation = 'softmax'))

cnn.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=lr), metrics=['accuracy'])

cnn.fit_generator(imagen_entrenamiento, steps_per_epoch=pasos, epochs=epocas, validation_data=imagen_validacion, validation_steps=pasos_validacion)

dir = r'C:\Users\JOSEA\Downloads\data\models'

if not os.path.exists(dir):
    os.mkdir(dir)
cnn.save(r'C:\Users\JOSEA\Downloads\data\models\modelo.h5')
cnn.save_weights(r'C:\Users\JOSEA\Downloads\data\models\pesos.h5')

Epoch 1/20
 3/64 [>.............................] - ETA: 1:48 - loss: 1.1976 - acc: 0.5000

KeyboardInterrupt: 